In [1]:
from pyspark import SparkConf, SparkContext
import string

In [2]:
if SparkContext._active_spark_context:
    SparkContext._active_spark_context.stop()



conf = SparkConf().setAppName("ExtendedWordCount").setMaster("local")
sc = SparkContext(conf=conf)

In [3]:
textFile = sc.textFile("a.txt")

Extended Word Count

In [4]:
wordCounts = textFile.flatMap(lambda line: line.split()) \
                    .map(lambda word: (word, 1)) \
                    .reduceByKey(lambda a, b: a + b)

sortedWordCounts = wordCounts.sortBy(lambda x: x[1], ascending=False)

results = sortedWordCounts.collect()

for word, count in results:
    print(f"{word}: {count}")


Streaming output truncated to the last 5000 lines.
act;: 1
trade: 1
board;: 1
Alien: 1
custodian: 1
(with: 1
duties);: 1
Agricultural: 1
stimulation;: 1
Housing: 1
construction: 1
(except: 1
ship-builders);: 1
telegraphs: 1
telephones;: 1
Export: 1
control.: 1
extended:: 1
railroads:: 1
Finance: 1
Corporation:: 1
liquidation.: 1
Capital: 1
Issues: 1
Committee:: 1
terminate: 1
proclamation.: 1
Aircraft: 1
Board:: 1
proclaimed;: 1
ships,: 1
officially: 1
Benjamin: 1
Eugene: 1
Clemenceau,: 1
world-famous: 1
Premier: 1
inspiring: 1
assaults: 1
enemy.]: 1
Wilson,: 1
democracies,: 1
arguments: 1
delegation: 1
conclave: 1
re-construction.: 1
delegates: 1
Robert: 1
Lansing,: 1
State;: 1
White,: 1
Tasker: 1
Bliss.: 1
Representing: 1
Peace: 1
Conference: 1
Samuel: 1
Gompers,: 1
Labor;: 1
Green,: 1
secretary-treasurer: 1
Mine: 1
Workers: 1
America;: 1
R.: 1
Alpine,: 1
Plumbers’: 1
Duncan,: 1
Association: 1
Granite: 1
Cutters;: 1
Duffy,: 1
Brotherhood: 1
Carpenters: 1
Joiners,: 1
Morrison,: 1
secr

In [5]:
import string

stopWords = set([
    "the", "and", "to", "of", "a", "in", "that", "is", "it", "was",
    "he", "for", "with", "as", "his", "on", "be", "at", "by", "i",
    "this", "not", "but", "are", "from", "or", "have", "an", "they",
    "you", "her", "she", "him", "their", "my", "we", "me", "so",
    "if", "them", "will", "would", "can", "all", "there", "what",
    "about", "up", "out", "who", "get", "which", "when", "make",
    "just", "like", "now", "than", "then", "some", "into", "could",
    "these", "no", "only", "its", "over", "also", "other", "how",
    "your", "because", "been", "do", "did", "didn't", "does", "didn’t", "were", "had"
])


In [6]:
def preprocess(line):
    translator = str.maketrans('', '', string.punctuation)
    return line.translate(translator).lower()


In [7]:

processedText = textFile.map(preprocess)

extendedWordCounts = processedText.flatMap(lambda line: line.split()) \
                                  .filter(lambda word: word not in stopWords) \
                                  .map(lambda word: (word, 1)) \
                                  .reduceByKey(lambda a, b: a + b)

sortedExtendedWordCounts = extendedWordCounts.sortBy(lambda x: x[1], ascending=False)

top15Words = sortedExtendedWordCounts.take(15)

print("\nTop 15 Most Common Words (After Cleaning):")
for word, count in top15Words:
    print(f"{word}: {count}")



Top 15 Most Common Words (After Cleaning):
german: 219
war: 196
against: 166
army: 160
germany: 158
french: 101
one: 100
general: 99
great: 97
first: 97
austria: 93
made: 90
project: 85
russia: 81
any: 79


In [8]:
!pip install pyngrok
from pyngrok import ngrok

ngrok.set_auth_token("2om1zYwDuD9fXJywSj2Jr2lCG3z_36d4WoZxEHzVQbveZye44")

public_url = ngrok.connect(4040)
print(f"Spark UI is accessible at: {public_url}")


Spark UI is accessible at: NgrokTunnel: "https://ad7b-34-19-76-174.ngrok-free.app" -> "http://localhost:4040"


# Part 2

In [9]:
from pyspark import SparkConf, SparkContext


if SparkContext._active_spark_context:
    SparkContext._active_spark_context.stop()


conf = SparkConf().setAppName("WordCountAnalysis").setMaster("local[*]")
sc = SparkContext(conf=conf)


In [10]:
textFile = sc.textFile("a.txt")

In [11]:
import string

stopWords = set([
    "the", "and", "to", "of", "a", "in", "that", "is", "it", "was",
    "he", "for", "with", "as", "his", "on", "be", "at", "by", "i",
    "this", "not", "but", "are", "from", "or", "have", "an", "they",
    "you", "her", "she", "him", "their", "my", "we", "me", "so",
    "if", "them", "will", "would", "can", "all", "there", "what",
    "about", "up", "out", "who", "get", "which", "when", "make",
    "just", "like", "now", "than", "then", "some", "into", "could",
    "these", "no", "only", "its", "over", "also", "other", "how",
    "your", "because", "been", "do", "did", "didn't", "does", "didn't", "were", "had"
])


In [12]:
def preprocess(line):
    translator = str.maketrans('', '', string.punctuation)
    return line.translate(translator).lower()


In [13]:
processedText = textFile.map(preprocess)


In [14]:
def generateBigrams(words):
    return zip(words, words[1:])


In [15]:
bigrams = processedText.flatMap(lambda line: generateBigrams(line.split()))
filteredBigrams = bigrams.filter(lambda bigram: bigram[0] not in stopWords and bigram[1] not in stopWords)
bigramPairs = filteredBigrams.map(lambda bigram: (bigram, 1))
bigramCounts = bigramPairs.reduceByKey(lambda a, b: a + b)
sortedBigramCounts = bigramCounts.sortBy(lambda x: x[1], ascending=False)
top10Bigrams = sortedBigramCounts.take(10)

In [16]:
print("\nTop 10 Most Frequent Bigrams:")
for bigram, count in top10Bigrams:
    print(f"{bigram[0]} {bigram[1]}: {count}")


Top 10 Most Frequent Bigrams:
project gutenberg™: 48
against germany: 41
united states: 39
project gutenberg: 21
army corps: 20
german army: 18
world war: 17
von hindenburg: 16
against austria: 16
electronic works: 16


In [17]:
from pyngrok import ngrok

ngrok.set_auth_token("2om1zYwDuD9fXJywSj2Jr2lCG3z_36d4WoZxEHzVQbveZye44")

public_url = ngrok.connect(4040)
print(f"Spark UI is accessible at: {public_url}")


Spark UI is accessible at: NgrokTunnel: "https://89b5-34-19-76-174.ngrok-free.app" -> "http://localhost:4040"
